In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon
from tqdm.notebook import tqdm
import time
from datetime import datetime

import requests
import re
from bs4 import BeautifulSoup
import json
import urllib


# отключить предупреждения пандас
pd.options.mode.chained_assignment = None

In [2]:
transp_reg = gpd.read_file('../data/from/transp_reg_st.shp',
                        encoding='utf-8')
# transp_reg.crs = 'epsg:53004'
transp_reg = transp_reg.to_crs('epsg:4326')

In [4]:
def find_js(lnk):
    with requests.get(lnk, stream=True, timeout=25) as req:
        bs = BeautifulSoup(req.text, 'html.parser')
        all_scrpt = bs.find_all("script")
        for scr in all_scrpt:
            if scr.get('type') == "application/json":
                sc_ind = all_scrpt.index(scr)
                break
            else:
                sc_ind=0
        # 
        scrp_txt = all_scrpt[sc_ind].text
        js = json.loads(scrp_txt)

        return js

In [5]:
def get_poi(word, srch_pt, poly):

    str_qt = urllib.parse.quote(word)

    coords1 = str(str(srch_pt.coords[0][0]) + "%2C" + str(srch_pt.coords[0][1]))
    coords2 = coords1

    url = "https://yandex.ru/maps/51/samara/search/{str_qt}/?ll={coords1}&sll={coords2}&sspn=0.015134%2C0.006711&z=15.96".format(str_qt=str_qt,
                                                                                                                                       coords1=coords1,
                                                                                                                                       coords2=coords2)
    # 
    js = find_js(url)

    fnd_items = js['searchPreloadedResults']['items']

    if len(fnd_items) == 0:
        lst_res = [word+'_not_found', None, srch_pt]
    else:
        i=0
        for i in range(len(fnd_items)):
            fnd_pt = Point(fnd_items[i]['coordinates'])
            if poly.intersects(fnd_pt):
                name = fnd_items[0]['title']
                lst_res = [word, name, fnd_pt]
                break
            else:
                lst_res = [word+'_not_found', None, srch_pt]
    # 
    return lst_res

In [6]:
def get_reg_centre(reg, lst_words):

    poly = reg.geometry[0]
    srch_pt = poly.centroid
    reg_id = reg.NO[0]

    bg_lst_found = []
    i=0
    for i in range(len(lst_words)):
        word = lst_words[i]
        try:
            lst_res = [reg_id] + get_poi(word, srch_pt, poly)
        except:
            time.sleep(30)
            lst_res = [reg_id] + get_poi(word, srch_pt, poly)
        bg_lst_found.append(lst_res)
    # 
    return bg_lst_found

In [10]:
lst_words = ['Торговый центр',
                'Железнодорожная станция',
                'Автостанция',
                'Площадь',
                'Больница',
                'ВУЗ',
                'Завод']

In [17]:
bg_lst_pts = []

i=0
# for i in tqdm(range(2)):
for i in tqdm(range(len(transp_reg))):
    reg = transp_reg.iloc[[i]].reset_index(drop=True)
    bg_lst_found = get_reg_centre(reg, lst_words)
    bg_lst_pts = bg_lst_pts + bg_lst_found
# 

In [18]:
clmns = ['reg_id', 'grp_pt', 'name_pt', 'geometry']
df = pd.DataFrame(bg_lst_pts, columns = clmns)
gdf_pts = gpd.GeoDataFrame(df, geometry='geometry')
gdf_pts.crs='epsg:4326'

In [19]:
gdf_pts.to_file('./data/res/gdf_pts_all.json', driver='GeoJSON', encoding='utf-8')

In [11]:
# gdf_pts_select = gpd.read_file('../data/res/gdf_pts_all_select.geojson',
#                         encoding='utf-8')